In [1]:
!pip install bertopic gensim sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
!wget https://files.consumerfinance.gov/ccdb/complaints.csv.zip -O data.zip

--2025-08-04 07:45:16--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 18.161.111.121, 18.161.111.66, 18.161.111.69, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|18.161.111.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1380614933 (1.3G) [binary/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]   1.29G  17.4MB/s    in 67s     

2025-08-04 07:46:24 (19.6 MB/s) - ‘data.zip’ saved [1380614933/1380614933]



In [ ]:
import zipfile

with zipfile.ZipFile("Complaints2.zip", 'r') as zip_ref:
    zip_ref.extractall("unzipped_data")

In [1]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import numpy as np
import os
import torch
from tqdm import tqdm
import random

# BERTopic Training

In [4]:
def load_sample_complaints(csv_path, col="Consumer complaint narrative", sample_size=20000, random_state=42):
    """
    Load a random sample of non-empty values from a CSV column.
    Will keep sampling until it gets the desired number of valid documents.
    """
    print(f"Loading data from {csv_path}...")

    # First, get the total number of rows to determine if we need to sample
    total_rows = sum(1 for line in open(csv_path)) - 1  # Subtract 1 for header
    print(f"Total rows in dataset: {total_rows}")

    # Strategy: Load chunks and filter until we have enough valid documents
    valid_texts = []
    chunk_size = max(sample_size * 3, 50000)  # Load 3x more to account for empty rows

    print(f"Target: {sample_size} valid documents")

    while len(valid_texts) < sample_size:
        remaining_needed = sample_size - len(valid_texts)
        print(f"Need {remaining_needed} more valid documents...")

        if total_rows <= chunk_size:
            # Load all data if dataset is small
            df = pd.read_csv(
                csv_path,
                usecols=[col],
                dtype={col: str},
                low_memory=False
            )
        else:
            # Sample a chunk
            sample_prob = min(chunk_size / total_rows, 1.0)
            df = pd.read_csv(
                csv_path,
                usecols=[col],
                dtype={col: str},
                low_memory=False,
                skiprows=lambda i: i > 0 and random.random() > sample_prob
            )

        print(f"Loaded chunk size: {len(df)} rows")

        # Clean and filter this chunk
        chunk_texts = df[col].dropna().str.strip()
        chunk_texts = chunk_texts[chunk_texts.astype(bool)].tolist()

        print(f"Valid documents in chunk: {len(chunk_texts)}")

        # Add to our collection, avoiding duplicates
        for text in chunk_texts:
            if text not in valid_texts:  # Simple duplicate check
                valid_texts.append(text)
                if len(valid_texts) >= sample_size:
                    break

        # If we've seen most of the dataset and still don't have enough, break
        if len(df) < chunk_size * 0.1:  # Very small chunk returned
            print("Warning: May have exhausted dataset before reaching target size")
            break

    # Shuffle and trim to exact size
    random.seed(random_state)
    random.shuffle(valid_texts)
    final_texts = valid_texts[:sample_size]

    print(f"Final dataset size: {len(final_texts)} documents")
    return final_texts

# Don't forget to import random at the top of your file
import random

# Configuration
csv_path = 'Complaints2.csv'
col_name = "Consumer complaint narrative"
sample_size = 20000

# Load sample data
print(f"Loading sample of {sample_size} complaints from {csv_path}...")
all_documents = load_sample_complaints(csv_path, col=col_name, sample_size=sample_size)

Loading sample of 20000 complaints from Complaints2.csv...
Loading data from Complaints2.csv...
Total rows in dataset: 960889
Target: 20000 valid documents
Need 20000 more valid documents...
Loaded chunk size: 41032 rows
Valid documents in chunk: 6232
Need 14643 more valid documents...
Loaded chunk size: 41091 rows
Valid documents in chunk: 6251
Need 9872 more valid documents...
Loaded chunk size: 41230 rows
Valid documents in chunk: 6253
Need 5552 more valid documents...
Loaded chunk size: 41268 rows
Valid documents in chunk: 6294
Need 1493 more valid documents...
Loaded chunk size: 41072 rows
Valid documents in chunk: 6424
Final dataset size: 20000 documents


In [5]:
# Initialize models
print("Initializing embedding model and BERTopic...")
print("GPU available:", "Yes" if torch.cuda.is_available() else "No")

# Initialize sentence transformer with GPU support
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
if torch.cuda.is_available():
    embedding_model = embedding_model.cuda()
    print("Embedding model moved to GPU")

topic_model = BERTopic(
    verbose=True,
    n_gram_range=(1, 2),
    min_topic_size=50,
    embedding_model=embedding_model,  # Pass embedding model to BERTopic
    calculate_probabilities=True  # This gives you topic probabilities for each document
)


# Train the model on the entire dataset

topics, probs = topic_model.fit_transform(all_documents)

print("Training complete!")
print(f"Number of topics discovered: {len(set(topics)) - 1}")  # -1 to exclude outlier topic (-1)
print(f"Number of documents processed: {len(all_documents)}")

# Save the model
model_path = "my_full_bertopic_model"
print(f"Saving model to {model_path}...")
topic_model.save(model_path, serialization="safetensors")
print(f"Model saved successfully to {model_path}")

# Optional: Display some basic topic information
print("\nTop 10 topics by size:")
topic_info = topic_model.get_topic_info()
print(topic_info.head(10))

# Optional: Save topic information to CSV
topic_info.to_csv("topic_info.csv", index=False)
print("Topic information saved to topic_info.csv")

print("\nTraining completed successfully!")
print("You can now load the model later using:")
print(f"loaded_model = BERTopic.load('{model_path}')")

Initializing embedding model and BERTopic...
GPU available: Yes


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2025-08-14 17:17:15,334 - BERTopic - Embedding - Transforming documents to embeddings.


Embedding model moved to GPU


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

2025-08-14 17:18:01,236 - BERTopic - Embedding - Completed ✓
2025-08-14 17:18:01,237 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-08-14 17:18:40,999 - BERTopic - Dimensionality - Completed ✓
2025-08-14 17:18:41,003 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-08-14 17:18:49,457 - BERTopic - Cluster - Completed ✓
2025-08-14 17:18:49,472 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-08-14 17:18:56,683 - BERTopic - Representation - Completed ✓


Training complete!
Number of topics discovered: 52
Number of documents processed: 20000
Saving model to my_full_bertopic_model...
Model saved successfully to my_full_bertopic_model

Top 10 topics by size:
   Topic  Count                                  Name  \
0     -1   5558              -1_xxxx_the_xxxx xxxx_to   
1      0   2066                     0_debt_the_to_and   
2      1   1027           1_consumer_15_xxxx xxxx_any   
3      2   1021    2_xxxx xxxx_xxxx_inquiries_inquiry   
4      3    960        3_accounts_my_report_my credit   
5      4    812  4_late_payment_late payment_payments   
6      5    659               5_xxxx_xxxx xxxx_the_of   
7      6    622                     6_the_was_to_bank   
8      7    487               7_loans_loan_to_student   
9      8    421                   8_equifax_to_my_the   

                                      Representation  \
0  [xxxx, the, xxxx xxxx, to, and, my, of, xx, cr...   
1  [debt, the, to, and, this, collection, of, tha...   

# Evaluate Model

In [6]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from scipy.spatial.distance import pdist, squareform
from scipy.stats import entropy
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

In [7]:
class BERTopicEvaluator:
    def __init__(self, model, documents, embeddings=None):
        """
        Initialize the evaluator with your trained BERTopic model

        Args:
            model: Trained BERTopic model
            documents: List of original documents used for training
            embeddings: Document embeddings (optional, will be computed if not provided)
        """
        self.model = model
        self.documents = documents
        self.embeddings = embeddings
        self.topics = model.get_topics()
        self.topic_labels = model.get_topic_info()

        # Get topic assignments for documents
        self.topic_assignments, _ = model.transform(documents)

        # Filter out outlier topic (-1)
        self.valid_docs = [i for i, topic in enumerate(self.topic_assignments) if topic != -1]
        self.valid_topics = [topic for topic in self.topic_assignments if topic != -1]

    def coherence_scores(self):
        """
        Calculate various coherence metrics (C_V, C_NPMI, C_UCI)
        """
        print("Computing coherence scores...")

        # Prepare texts for coherence calculation
        texts = [doc.lower().split() for doc in self.documents]
        dictionary = Dictionary(texts)

        # Debug: Let's see what we're working with
        print("Debug: Checking topic format...")
        sample_topic = self.model.get_topic(0)
        print(f"Sample topic format: {type(sample_topic)}")
        print(f"First few items: {sample_topic[:3]}")
        print(f"First item type: {type(sample_topic[0])}")

        # Get topic words - simplified approach
        topic_words = []
        topic_info = self.model.get_topic_info()

        # Get valid topic IDs (excluding outlier topic -1)
        valid_topic_ids = [tid for tid in topic_info['Topic'].tolist() if tid >= 0]
        print(f"Valid topic IDs: {len(valid_topic_ids)}")

        for topic_id in valid_topic_ids:
            try:
                topic_terms = self.model.get_topic(topic_id)
                print(f"\nTopic {topic_id} raw terms: {topic_terms[:3]}")

                # Simple extraction - just get the first element of each term
                words = []
                for i, term in enumerate(topic_terms[:10]):
                    try:
                        word = term[0]  # Get first element (should be word)
                        words.append(str(word))
                        if i < 3:  # Debug first few
                            print(f"  Extracted word {i}: '{word}' (type: {type(word)})")
                    except Exception as e:
                        print(f"  Error extracting word {i}: {e}")
                        continue

                if words:
                    topic_words.append(words)
                    print(f"  Final words for topic {topic_id}: {words[:5]}")

            except Exception as e:
                print(f"Error processing topic {topic_id}: {e}")
                continue

        print(f"\nExtracted words from {len(topic_words)} topics")

        # Test with just one coherence metric first
        coherence_scores = {}

        if not topic_words:
            return {'C_V': None, 'C_NPMI': None, 'C_UCI': None}

        # Debug: Check the final format
        print(f"Topic words format check:")
        print(f"  Number of topics: {len(topic_words)}")
        print(f"  First topic words: {topic_words[0] if topic_words else 'None'}")
        print(f"  Word types: {[type(w) for w in topic_words[0][:3]] if topic_words else 'None'}")

        # Try C_V coherence only first
        try:
            coherence_model_cv = CoherenceModel(
                topics=topic_words,
                texts=texts,
                dictionary=dictionary,
                coherence='c_v'
            )
            coherence_scores['C_V'] = coherence_model_cv.get_coherence()
            print("C_V coherence calculated successfully!")
        except Exception as e:
            print(f"Error calculating C_V coherence: {e}")
            print(f"Error type: {type(e)}")
            coherence_scores['C_V'] = None

        # If C_V works, try the others
        if coherence_scores['C_V'] is not None:
            try:
                coherence_model_npmi = CoherenceModel(
                    topics=topic_words,
                    texts=texts,
                    dictionary=dictionary,
                    coherence='c_npmi'
                )
                coherence_scores['C_NPMI'] = coherence_model_npmi.get_coherence()
            except Exception as e:
                print(f"Error calculating C_NPMI coherence: {e}")
                coherence_scores['C_NPMI'] = None

            try:
                coherence_model_uci = CoherenceModel(
                    topics=topic_words,
                    texts=texts,
                    dictionary=dictionary,
                    coherence='c_uci'
                )
                coherence_scores['C_UCI'] = coherence_model_uci.get_coherence()
            except Exception as e:
                print(f"Error calculating C_UCI coherence: {e}")
                coherence_scores['C_UCI'] = None
        else:
            coherence_scores['C_NPMI'] = None
            coherence_scores['C_UCI'] = None

        return coherence_scores

    def clustering_metrics(self):
        """
        Calculate clustering quality metrics using document embeddings
        """
        print("Computing clustering metrics...")

        if self.embeddings is None:
            print("Warning: No embeddings provided. Computing embeddings...")
            # If embeddings not provided, use the model's sentence transformer
            self.embeddings = self.model._extract_embeddings(
                self.documents,
                method="sentence-transformers"
            )

        # Filter embeddings for valid topics only
        valid_embeddings = self.embeddings[self.valid_docs]

        clustering_scores = {}

        # Silhouette Score
        try:
            silhouette = silhouette_score(valid_embeddings, self.valid_topics)
            clustering_scores['silhouette_score'] = silhouette
        except Exception as e:
            print(f"Error calculating silhouette score: {e}")
            clustering_scores['silhouette_score'] = None

        # Calinski-Harabasz Index
        try:
            ch_score = calinski_harabasz_score(valid_embeddings, self.valid_topics)
            clustering_scores['calinski_harabasz_score'] = ch_score
        except Exception as e:
            print(f"Error calculating Calinski-Harabasz score: {e}")
            clustering_scores['calinski_harabasz_score'] = None

        # Davies-Bouldin Index (lower is better)
        try:
            db_score = davies_bouldin_score(valid_embeddings, self.valid_topics)
            clustering_scores['davies_bouldin_score'] = db_score
        except Exception as e:
            print(f"Error calculating Davies-Bouldin score: {e}")
            clustering_scores['davies_bouldin_score'] = None

        return clustering_scores

    def topic_diversity_metrics(self):
        """
        Calculate topic diversity metrics
        """
        print("Computing topic diversity metrics...")

        diversity_scores = {}

        # Get top words for each topic
        all_topic_words = []
        for topic_id in range(len(self.topics)):
            if topic_id in self.topics:
                words = [word for word, _ in self.model.get_topic(topic_id)[:10]]
                all_topic_words.extend(words)

        # Topic Diversity (TD) - proportion of unique words across topics
        unique_words = set(all_topic_words)
        total_words = len(all_topic_words)
        diversity_scores['topic_diversity'] = len(unique_words) / total_words if total_words > 0 else 0

        # Pairwise topic diversity (average Jaccard distance between topics)
        topic_words_sets = []
        for topic_id in range(len(self.topics)):
            if topic_id in self.topics:
                words = set([word for word, _ in self.model.get_topic(topic_id)[:10]])
                topic_words_sets.append(words)

        if len(topic_words_sets) > 1:
            jaccard_distances = []
            for i in range(len(topic_words_sets)):
                for j in range(i + 1, len(topic_words_sets)):
                    intersection = len(topic_words_sets[i] & topic_words_sets[j])
                    union = len(topic_words_sets[i] | topic_words_sets[j])
                    jaccard_sim = intersection / union if union > 0 else 0
                    jaccard_distances.append(1 - jaccard_sim)

            diversity_scores['avg_pairwise_jaccard_distance'] = np.mean(jaccard_distances)
        else:
            diversity_scores['avg_pairwise_jaccard_distance'] = 0

        return diversity_scores

    def topic_quality_metrics(self):
        """
        Calculate additional topic quality metrics
        """
        print("Computing topic quality metrics...")

        quality_scores = {}

        # Number of topics
        quality_scores['num_topics'] = len([t for t in self.topics.keys() if t >= 0])

        # Coverage (percentage of documents assigned to topics, excluding outliers)
        total_docs = len(self.documents)
        assigned_docs = len(self.valid_docs)
        quality_scores['coverage'] = assigned_docs / total_docs

        # Topic size distribution metrics
        topic_sizes = Counter(self.valid_topics)
        sizes = list(topic_sizes.values())

        if sizes:
            quality_scores['avg_topic_size'] = np.mean(sizes)
            quality_scores['std_topic_size'] = np.std(sizes)
            quality_scores['topic_size_entropy'] = entropy(sizes)  # Higher = more balanced
            quality_scores['largest_topic_ratio'] = max(sizes) / sum(sizes)

        # Topic probability distribution quality
        try:
            # Get topic probabilities for documents
            topics_probs = self.model.probabilities_
            if topics_probs is not None:
                # Average confidence (higher = more confident assignments)
                max_probs = np.max(topics_probs, axis=1)
                quality_scores['avg_confidence'] = np.mean(max_probs)

                # Probability distribution entropy (lower = more confident)
                prob_entropies = [entropy(prob_dist) for prob_dist in topics_probs]
                quality_scores['avg_prob_entropy'] = np.mean(prob_entropies)
        except:
            quality_scores['avg_confidence'] = None
            quality_scores['avg_prob_entropy'] = None

        return quality_scores

    def perplexity_score(self, sample_size=1000):
        """
        Calculate perplexity-like score for topic model
        """
        print("Computing perplexity score...")

        try:
            # Sample documents for efficiency
            if len(self.documents) > sample_size:
                sample_indices = np.random.choice(len(self.documents), sample_size, replace=False)
                sample_docs = [self.documents[i] for i in sample_indices]
            else:
                sample_docs = self.documents

            # Get topic probabilities
            _, probs = self.model.transform(sample_docs)

            if probs is not None:
                # Calculate log-likelihood
                log_likelihood = 0
                for prob_dist in probs:
                    # Avoid log(0) by adding small epsilon
                    prob_dist = prob_dist + 1e-10
                    log_likelihood += np.log(np.max(prob_dist))

                # Calculate perplexity
                perplexity = np.exp(-log_likelihood / len(sample_docs))
                return perplexity
            else:
                return None
        except Exception as e:
            print(f"Error calculating perplexity: {e}")
            return None

    def evaluate_all(self, save_results=True, plot_results=True):
        """
        Run all evaluation metrics and compile results
        """
        print("Starting comprehensive evaluation...")
        print("="*50)

        results = {}

        # Coherence scores
        results['coherence'] = self.coherence_scores()

        # Clustering metrics
        results['clustering'] = self.clustering_metrics()

        # Topic diversity
        results['diversity'] = self.topic_diversity_metrics()

        # Topic quality
        results['quality'] = self.topic_quality_metrics()

        # Perplexity
        results['perplexity'] = self.perplexity_score()

        # Print results
        self.print_results(results)

        if plot_results:
            self.plot_evaluation_results(results)

        if save_results:
            self.save_results_to_csv(results)

        return results

    def print_results(self, results):
        """
        Print formatted evaluation results
        """
        print("\n" + "="*50)
        print("BERTOPIC MODEL EVALUATION RESULTS")
        print("="*50)

        print("\n📊 COHERENCE METRICS:")
        for metric, score in results['coherence'].items():
            if score is not None:
                print(f"  {metric}: {score:.4f}")
            else:
                print(f"  {metric}: N/A")

        print("\n🎯 CLUSTERING METRICS:")
        for metric, score in results['clustering'].items():
            if score is not None:
                print(f"  {metric}: {score:.4f}")
            else:
                print(f"  {metric}: N/A")

        print("\n🌈 DIVERSITY METRICS:")
        for metric, score in results['diversity'].items():
            print(f"  {metric}: {score:.4f}")

        print("\n⭐ QUALITY METRICS:")
        for metric, score in results['quality'].items():
            if score is not None:
                if isinstance(score, float):
                    print(f"  {metric}: {score:.4f}")
                else:
                    print(f"  {metric}: {score}")
            else:
                print(f"  {metric}: N/A")

        if results['perplexity'] is not None:
            print(f"\n🔍 PERPLEXITY: {results['perplexity']:.4f}")

        print("\n" + "="*50)
        print("INTERPRETATION GUIDELINES:")
        print("📊 Coherence (C_V): Higher is better (typical range: 0.3-0.7)")
        print("🎯 Silhouette Score: Higher is better (range: -1 to 1)")
        print("🎯 Davies-Bouldin: Lower is better")
        print("🌈 Topic Diversity: Higher is better (range: 0-1)")
        print("⭐ Coverage: Higher is better (range: 0-1)")
        print("🔍 Perplexity: Lower is better")
        print("="*50)

    def plot_evaluation_results(self, results):
        """
        Create visualizations of evaluation results
        """
        print("\nGenerating evaluation plots...")

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('BERTopic Model Evaluation Results', fontsize=16, fontweight='bold')

        # Plot 1: Coherence Scores
        coherence_data = {k: v for k, v in results['coherence'].items() if v is not None}
        if coherence_data:
            axes[0, 0].bar(coherence_data.keys(), coherence_data.values(), color='skyblue')
            axes[0, 0].set_title('Coherence Scores')
            axes[0, 0].set_ylabel('Score')
            axes[0, 0].tick_params(axis='x', rotation=45)

        # Plot 2: Clustering Metrics
        clustering_data = {k: v for k, v in results['clustering'].items() if v is not None}
        if clustering_data:
            axes[0, 1].bar(clustering_data.keys(), clustering_data.values(), color='lightcoral')
            axes[0, 1].set_title('Clustering Quality Metrics')
            axes[0, 1].set_ylabel('Score')
            axes[0, 1].tick_params(axis='x', rotation=45)

        # Plot 3: Topic Size Distribution
        topic_sizes = Counter(self.valid_topics)
        if topic_sizes:
            axes[1, 0].hist(topic_sizes.values(), bins=20, alpha=0.7, color='lightgreen')
            axes[1, 0].set_title('Topic Size Distribution')
            axes[1, 0].set_xlabel('Topic Size')
            axes[1, 0].set_ylabel('Frequency')

        # Plot 4: Quality Metrics Summary
        quality_metrics = ['coverage', 'topic_diversity', 'avg_confidence']
        quality_values = []
        quality_labels = []

        for metric in quality_metrics:
            if metric in results['quality'] and results['quality'][metric] is not None:
                quality_values.append(results['quality'][metric])
                quality_labels.append(metric.replace('_', ' ').title())

        if quality_values:
            axes[1, 1].bar(quality_labels, quality_values, color='gold')
            axes[1, 1].set_title('Key Quality Metrics')
            axes[1, 1].set_ylabel('Score')
            axes[1, 1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()

    def save_results_to_csv(self, results):
        """
        Save evaluation results to CSV file
        """
        print("Saving results to CSV...")

        # Flatten results dictionary
        flattened_results = {}
        for category, metrics in results.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    flattened_results[f"{category}_{metric}"] = value
            else:
                flattened_results[category] = metrics

        # Create DataFrame and save
        df = pd.DataFrame([flattened_results])
        df.to_csv('bertopic_evaluation_results.csv', index=False)
        print("Results saved to 'bertopic_evaluation_results.csv'")




In [11]:
# Display results
def display_results(coherence_scores, diversity_scores, quality_scores, perplexity):
    print("="*50)
    print("BERTOPIC EVALUATION RESULTS")
    print("="*50)

    print("\n📊 COHERENCE SCORES:")
    for metric, score in coherence_scores.items():
        if score is not None:
            print(f"  {metric}: {score:.4f}")
        else:
            print(f"  {metric}: N/A")

    print("\n🌈 DIVERSITY SCORES:")
    for metric, score in diversity_scores.items():
        print(f"  {metric}: {score:.4f}")

    print("\n⭐ QUALITY SCORES:")
    for metric, score in quality_scores.items():
        if score is not None:
            if isinstance(score, float):
                print(f"  {metric}: {score:.4f}")
            else:
                print(f"  {metric}: {score}")
        else:
            print(f"  {metric}: N/A")

    if perplexity is not None:
        print(f"\n🔍 PERPLEXITY: {perplexity:.4f}")
    else:
        print(f"\n🔍 PERPLEXITY: N/A")

    print("\n" + "="*50)
    print("QUICK INTERPRETATION:")
    print("📊 C_V Coherence > 0.4 = Good, > 0.5 = Excellent")
    print("🌈 Topic Diversity > 0.7 = Good topic separation")
    print("⭐ Coverage > 0.8 = Most documents well-classified")
    print("🔍 Lower Perplexity = Better model fit")
    print("="*50)

In [ ]:
def calculate_coherence_scores(model, documents):
    """
    Calculates various coherence metrics for a BERTopic model.

    This function is designed to work as a standalone utility.

    Args:
        model: A trained BERTopic model instance.
        documents (list of str): The list of documents used to train the model.

    Returns:
        dict: A dictionary containing the calculated coherence scores
              for 'C_V', 'C_NPMI', and 'C_UCI'. Returns None for any
              score that could not be calculated.
    """
    print("Computing coherence scores for the topic model...")

    # 1. Prepare the corpus and dictionary for Gensim
    texts = [doc.lower().split() for doc in documents]
    dictionary = Dictionary(texts)

    # 2. Extract top words from each topic
    topics = []
    valid_topic_ids = [topic_id for topic_id in model.get_topics().keys() if topic_id != -1]

    for topic_id in valid_topic_ids[:10]:
        topic_as_tuples = model.get_topic(topic_id)

        # --- THIS IS THE FIX ---
        # Only add the topic if it's not empty
        if topic_as_tuples:
            topic_as_words = [word for word, score in topic_as_tuples]
            topics.append(topic_as_words)

    if not topics:
        print("Warning: No valid, non-empty topics were found to calculate coherence.")
        return {'C_V': None, 'C_NPMI': None, 'C_UCI': None}
    print("Verifying topics format...")
    print(topics)

    # 3. Calculate the coherence scores using Gensim's CoherenceModel
    coherence_scores = {}

    # C_V Coherence
    try:
        cv_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_scores['C_V'] = cv_model.get_coherence()
    except Exception as e:
        print(f"Error calculating C_V coherence: {e}")
        coherence_scores['C_V'] = None

    # C_NPMI Coherence
    try:
        npmi_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_npmi')
        coherence_scores['C_NPMI'] = npmi_model.get_coherence()
    except Exception as e:
        print(f"Error calculating C_NPMI coherence: {e}")
        coherence_scores['C_NPMI'] = None

    # C_UCI Coherence
    try:
        uci_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_uci')
        coherence_scores['C_UCI'] = uci_model.get_coherence()
    except Exception as e:
        print(f"Error calculating C_UCI coherence: {e}")
        coherence_scores['C_UCI'] = None

    print(f"✅ Coherence scores calculated: {coherence_scores}")
    return coherence_scores


In [8]:
# Load Model
loaded_model = BERTopic.load("topic_model", embedding_model=SentenceTransformer("all-MiniLM-L6-v2"))


ValueError: Make sure to either pass a valid directory or HF model.

In [9]:

# Create evaluator
evaluator = BERTopicEvaluator(topic_model, all_documents)
# run metrics
coherence_scores = evaluator.coherence_scores()
diversity_scores = evaluator.topic_diversity_metrics()
quality_scores = evaluator.topic_quality_metrics()
perplexity = evaluator.perplexity_score()

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

2025-08-14 17:23:04,692 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-08-14 17:23:04,748 - BERTopic - Dimensionality - Completed ✓
2025-08-14 17:23:04,748 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-08-14 17:23:06,015 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-08-14 17:23:12,360 - BERTopic - Probabilities - Completed ✓
2025-08-14 17:23:12,361 - BERTopic - Cluster - Completed ✓


Computing coherence scores...
Debug: Checking topic format...
Sample topic format: <class 'list'>
First few items: [('debt', 0.02222151069206836), ('the', 0.014376462035878775), ('to', 0.014093826720324948)]
First item type: <class 'tuple'>
Valid topic IDs: 52

Topic 0 raw terms: [('debt', 0.02222151069206836), ('the', 0.014376462035878775), ('to', 0.014093826720324948)]
  Extracted word 0: 'debt' (type: <class 'str'>)
  Extracted word 1: 'the' (type: <class 'str'>)
  Extracted word 2: 'to' (type: <class 'str'>)
  Final words for topic 0: ['debt', 'the', 'to', 'and', 'this']

Topic 1 raw terms: [('consumer', 0.025217303135384995), ('15', 0.01961228553646363), ('xxxx xxxx', 0.01747629774385948)]
  Extracted word 0: 'consumer' (type: <class 'str'>)
  Extracted word 1: '15' (type: <class 'str'>)
  Extracted word 2: 'xxxx xxxx' (type: <class 'str'>)
  Final words for topic 1: ['consumer', '15', 'xxxx xxxx', 'any', 'of']

Topic 2 raw terms: [('xxxx xxxx', 0.04602263793427051), ('xxxx', 0.03

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2025-08-14 17:26:36,708 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-08-14 17:26:51,798 - BERTopic - Dimensionality - Completed ✓
2025-08-14 17:26:51,799 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-08-14 17:26:51,876 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-08-14 17:26:52,249 - BERTopic - Probabilities - Completed ✓
2025-08-14 17:26:52,250 - BERTopic - Cluster - Completed ✓


In [ ]:
print(calculate_coherence_scores(loaded_model, all_documents))

Computing coherence scores for the topic model...
Verifying topics format...
[['mortgage', 'we', 'loan', 'our', 'escrow', 'was', 'home', 'xx', 'they', 'insurance'], ['wells', 'fargo', 'wells fargo', 'was', 'with wells', 'they', 'me', 'had', 'bank', '00'], ['chase', 'chase bank', 'bank', 'was', 'card', 'check', 'they', 'me', 'had', 'with chase'], ['bank', 'bank of', 'check', 'of america', 'america', 'funds', 'money', 'the bank', 'was', 'my account'], ['experian', 'experian has', 'dispute', 'experian is', 'against experian', 'to experian', 'credit', 'that experian', 'fcra', 'disputes'], ['synchrony', 'interest', 'card', 'synchrony bank', 'payment', 'fee', 'was', 'bank', 'balance', '00'], ['fcra', 'off', 'charge off', 'account', 'inaccurate', 'reporting', 'dispute', 'reported', 'charge', 'credit'], ['therefore', '719 pressume', 'sw 715', '281 sw', '715 719', '715', 'sw', 'mo 281', '281', '719'], ['debt', 'validation', 'collection', 'the debt', 'this debt', 'fdcpa', 'alleged', 'debt collec

In [12]:
display_results(coherence_scores, diversity_scores, quality_scores, perplexity)

BERTOPIC EVALUATION RESULTS

📊 COHERENCE SCORES:
  C_V: 0.5643
  C_NPMI: 0.0699
  C_UCI: 0.2519

🌈 DIVERSITY SCORES:
  topic_diversity: 0.4538
  avg_pairwise_jaccard_distance: 0.9016

⭐ QUALITY SCORES:
  num_topics: 52
  coverage: 0.7221
  avg_topic_size: 277.7308
  std_topic_size: 352.6516
  topic_size_entropy: 3.4317
  largest_topic_ratio: 0.1431
  avg_confidence: 0.3497
  avg_prob_entropy: 2.6425

🔍 PERPLEXITY: 3.5606

QUICK INTERPRETATION:
📊 C_V Coherence > 0.4 = Good, > 0.5 = Excellent
🌈 Topic Diversity > 0.7 = Good topic separation
⭐ Coverage > 0.8 = Most documents well-classified
🔍 Lower Perplexity = Better model fit
